In [12]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

In [13]:
import numpy as np
data = {}
def convert_string_int(s):
    rs = s.split(" ")
    rs = [int(x) for x in rs]
    return rs

def load_data(file_path):

    with open(file=file_path) as f:
        line1 = f.readline()
        data['N'], data['D'] = convert_string_int(line1)
        data['w'] = np.ones((data['N']+1,data['N']+1,data['D']+1))*99
        line2 = f.readline()
        data['s'], data['t'] = convert_string_int(line2)
        lines = f.readlines()
        for line in lines:
            s,t,w,c = convert_string_int(line)
            data['w'][s][t][c] = min(data['w'][s][t][c],w)
    f.close()



In [14]:
fpath =  'MFEA_lib/tasks/Benchmark/__references__/IDPC_DU/IDPC_EDU/data/set2/idpc_50x25x38961.idpc'
load_data(fpath)
data['w'][1]

array([[99., 99., 99., ..., 99., 99., 99.],
       [99., 99., 99., ..., 99., 99., 99.],
       [99., 99., 58., ..., 99., 99., 99.],
       ...,
       [99., 70., 99., ..., 99., 99., 99.],
       [99., 99., 99., ..., 99., 82., 99.],
       [99., 74., 84., ..., 93., 56., 99.]])

In [15]:
from tkinter import N


x = {}
n = data['N']
d = data['D']
for i in range(1, n + 1):
    for j in range(1, n + 1):
        for k in range(1,d+1):
            x[str(i) + "," + str(j) + "," + str(k)] = model.NewIntVar(0, 1, "x[" + str(i) + "," + str(j)+ "," + str(k) + "]")

t = {}
for i in range(1,n+1):
    for j in range(1,d+1):
        t[str(i) + "," + str(j)] = model.NewIntVar(0, 1, "t[" + str(i) + "," + str(j) + "]")

p = {}

for i in range(1,d+1):
    p[str(i)] = model.NewIntVar(0,d, "p[" + str(i) + "]")



In [16]:
for i in range(1,n+1):
    in_deg = x[str(1) + "," + str(i) + "," + str(1)]*1
    out_deg = x[str(i) + "," + str(1) + "," + str(1)]*1
    for j in range(1,n+1):
        for k in range(1,d+1):
            if j == 1 and k == 1: 
                continue
            in_deg += x[str(j) + "," + str(i)+ "," + str(k)]
            out_deg += x[str(i) + "," + str(j)+"," + str(k)]
    if i == data['s']: 
        model.Add(in_deg == 0)
        model.Add(out_deg == 1)
    elif i == data['t']:
        model.Add(in_deg == 1)
        model.Add(out_deg == 0)
    else:
        model.Add(in_deg == out_deg)
        model.Add(in_deg <=1)
        model.Add(out_deg <= 1)

    

In [17]:
for k in range(1,d+1):
    exp = x[str(1) + ","+str(1)+","+str(k)]*1
    for i in range(1,n+1):
        for j in range(1,n+1):  
            exp += x[str(1) + ","+str(1)+","+str(k)]
    model.Add(p[str(k)] == exp)

for i in range(1,n+1):
    for k in range(1,d+1):
        exp = x[str(i) + ","+str(1)+","+str(k)]*1
        exp+=x[str(1) + ","+str(i)+","+str(k)]*1
        a = model.NewIntVar(0,1,"a")
        model.AddMultiplicationEquality(a,[x[str(i) + ","+str(1)+","+str(k)],x[str(1) + ","+str(i)+","+str(k)]])
        exp-=a*1
        for j in range(2,n+1):
            exp += x[str(i) + ","+str(j)+","+str(k)]
            exp += x[str(j) + ","+str(i)+","+str(k)]
            b = model.NewIntVar(0,1,"b")
            model.AddMultiplicationEquality(a,[x[str(i) + ","+str(j)+","+str(k)],x[str(j) + ","+str(i)+","+str(k)]])
            exp-= b*1
        model.Add(exp == t[str(i)+","+str(k)])

In [18]:
for k in range(1,d+1):
    exp = t[str(1)+","+str(k)]*1
    for i in range(2,n+1):
        exp+= t[str(i)+","+str(k)]
    model.Add(exp - p[str(k)] <= 1)


In [19]:
func = model.NewIntVar(0,1000,"func")
for i in range(1,n+1):
    for j in range(1,n+1):
        for k in range(1,d+1):
            func+= x[str(i) + ","+str(j)+","+str(k)] * data['w'][i][j][k]
model.Minimize(func)
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(solver.ObjectiveValue())

print('\nStatistics')
print(f'  status   : {solver.StatusName(status)}')
print(f'  conflicts: {solver.NumConflicts()}')
print(f'  branches : {solver.NumBranches()}')
print(f'  wall time: {solver.WallTime()} s')

38.0

Statistics
  status   : OPTIMAL
  conflicts: 2
  branches : 108482
  wall time: 32.2779407 s
